<a href="https://colab.research.google.com/github/OliverWills/Cars/blob/main/BERT_cars_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
class BertRegressor(nn.Module):

    def __init__(self, drop_rate=0.2, freeze_bert=False):

        super(BertRegressor, self).__init__()
        D_in, D_out = 768, 1

        self.bert = \
                   BertModel.from_pretrained('bert-base-cased')
        self.regressor = nn.Sequential(
            nn.Dropout(drop_rate),
            nn.Linear(D_in, D_out))
    def forward(self, input_ids, attention_masks):

        outputs = self.bert(input_ids, attention_masks)
        class_label_output = outputs[1]
        outputs = self.regressor(class_label_output)
        return outputs

model = BertRegressor(drop_rate=0.2)

In [ ]:
model = torch.load(PATH)
model.eval()

In [ ]:
# Mount Google Drive and read data from CSV file
drive.mount('/content/drive')
data = pd.read_csv("/content/drive/My Drive/BMW_cleansed.csv")

In [ ]:
data['id']=data['Link'].str.split('/').str[-1]
data['age']=datetime.datetime.now().year-data['Year']
data['combined']=data['id']+\
  " This is "+data['Title']+\
  ". It is located in "+data['Location']+\
  ". The car is "+data['age'].astype(str)+\
  " years old. It has an engine size of "+data['Engine Size'].astype(str)+\
  "cc. It uses "+data['Fuel Type']+\
  " fuel. It has done "+data['Mileage'].astype(str)+\
  " miles. "+data['Description']
data=data.dropna()
Q1, Q3 = data.Price.quantile([0.25,0.75])
IQR = Q3 - Q1
limit = Q3 + 1.5 * IQR
data=data[data['Price']<limit]
data = data[['Price','combined']]
descriptions=data['combined'].tolist()

In [ ]:
# Download and define tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# encode descriptions using tokenizer
encoded_corpus = tokenizer(text=descriptions,
  add_special_tokens=True,
  padding='max_length',
  truncation='longest_first',
  max_length=512,
  return_attention_mask=True)

In [ ]:
# Define input ids and attention masks
input_ids = encoded_corpus['input_ids']
attention_mask = encoded_corpus['attention_mask']

input_ids = np.array(input_ids)
attention_mask = np.array(attention_mask)
labels = data['Price'].to_numpy()

In [ ]:
# Split train and test inputs and labels
test_size = 0.1
seed = 42

train_valid_inputs, test_inputs, train_valid_labels, test_labels = \
            train_test_split(input_ids, labels, test_size=test_size,
                             random_state=seed)

train_inputs, valid_inputs, train_labels, valid_labels = \
            train_test_split(train_valid_inputs, train_valid_labels, test_size=test_size,
                             random_state=seed)

train_valid_masks, test_masks, _, _ = train_test_split(attention_mask,
                                        labels, test_size=test_size,
                                        random_state=seed)

train_masks, valid_masks, _, _ = train_test_split(train_valid_masks,
                                        train_valid_labels, test_size=test_size,
                                        random_state=seed)

In [ ]:
# Define and create train and test dataloaders
batch_size = 16
def create_dataloaders(inputs, masks, labels, batch_size):
    input_tensor = torch.tensor(inputs)
    mask_tensor = torch.tensor(masks)
    labels_tensor = torch.tensor(labels)
    dataset = TensorDataset(input_tensor, mask_tensor,
                            labels_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size,
                            shuffle=True)
    return dataloader

In [ ]:
test_dataloader = create_dataloaders(test_inputs, test_masks,
                                     test_labels, batch_size)

In [ ]:
def predict(model, dataloader, device):
    model.eval()
    output = []
    for batch in dataloader:
        batch_inputs, batch_masks, _ = \
                                  tuple(b.to(device) for b in batch)
        with torch.no_grad():
            output += model(batch_inputs,
                            batch_masks).view(1,-1).tolist()[0]
    return output

In [ ]:
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
y_pred_scaled = predict(model, test_dataloader, device)

In [ ]:
y_test = np.array(test_labels)
y_pred_scaled = np.array(y_pred_scaled)

y_pred = price_scaler.inverse_transform(y_pred_scaled.reshape(-1,1))

In [ ]:
plt.scatter(y_test, y_pred)
plt.xlabel('y test')
plt.ylabel('y pred')
plt.show()

In [ ]:
results_df = pd.DataFrame(y_pred, columns=['y_pred'])
results_df['y_test']=pd.Series(y_test)
results_df['price_diff']=results_df['y_pred']-results_df['y_test']
results_df.sort_values('price_diff', axis=0, ascending=False, inplace=True)
print(results_df.head(10))

In [ ]:
test_inputs
test_labels